# 🔬 Phase 2: NLI-Tuned Encoder

## Finding from Phase 1
Baseline A (float, no HDC) = 54.6% — the encoder/features are the bottleneck!

## Hypothesis
NLI-tuned encoders should provide better relational features.

## Encoders to Test
| Encoder | Training | Size |
|---------|----------|------|
| all-MiniLM-L6-v2 (baseline) | General similarity | 384d |
| nli-distilroberta-base-v2 | SNLI + MNLI | 768d |
| nli-mpnet-base-v2 | AllNLI | 768d |

---

In [ ]:
!pip install -q sentence-transformers datasets
print("✅ Dependencies installed")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import json

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

print(f"PyTorch: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print(f"\n🔬 Phase 2: NLI-Tuned Encoder")

In [ ]:
# Load MNLI
print("Loading MNLI...")
dataset = load_dataset("glue", "mnli")

TRAIN_SIZE = 5000
TEST_SIZE = 500

train_data = dataset['train'].shuffle(seed=42).select(range(TRAIN_SIZE))
test_data = dataset['validation_matched'].select(range(TEST_SIZE))

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

train_premises = list(train_data['premise'])
train_hypotheses = list(train_data['hypothesis'])
test_premises = list(test_data['premise'])
test_hypotheses = list(test_data['hypothesis'])

print(f"✅ Train: {TRAIN_SIZE}, Test: {TEST_SIZE}")

In [ ]:
# Encoders to test
ENCODERS = {
    'MiniLM (baseline)': 'all-MiniLM-L6-v2',
    'NLI-DistilRoBERTa': 'sentence-transformers/nli-distilroberta-base-v2',
    'NLI-MPNet': 'sentence-transformers/nli-mpnet-base-v2',
}

print("📋 Encoders to test:")
for name, model_name in ENCODERS.items():
    print(f"   {name}: {model_name}")

In [ ]:
# Dataset and Model
class SimpleDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, num_classes=3, dropout=0.3):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

In [ ]:
def make_pair_features(p, h):
    """Create pair features: [P, H, P-H, P*H]"""
    return np.concatenate([p, h, p - h, p * h], axis=1)

def random_projection(features, dim=4096, seed=42):
    """Random projection to higher dimension."""
    np.random.seed(seed)
    proj = np.random.randn(features.shape[1], dim).astype(np.float32)
    proj /= np.linalg.norm(proj, axis=0, keepdims=True)
    return features @ proj

def ternary_quantize(features):
    """Quantize to {-1, 0, +1}."""
    thr = 0.3 * np.std(features, axis=1, keepdims=True)
    return np.where(features > thr, 1,
                    np.where(features < -thr, -1, 0)).astype(np.float32)

def train_and_evaluate(train_features, train_labels, test_features, test_labels,
                       input_dim, num_epochs=25, lr=1e-3):
    """Train MLP and return best accuracy."""
    
    train_dataset = SimpleDataset(train_features, train_labels)
    test_dataset = SimpleDataset(test_features, test_labels)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    model = MLPClassifier(input_dim=input_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    best_acc = 0
    
    for epoch in range(num_epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = criterion(model(features), labels)
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        model.eval()
        all_preds = []
        with torch.no_grad():
            for features, _ in test_loader:
                features = features.to(device)
                preds = torch.argmax(model(features), dim=1)
                all_preds.extend(preds.cpu().numpy())
        
        acc = accuracy_score(test_labels, all_preds)
        best_acc = max(best_acc, acc)
    
    return best_acc

In [ ]:
print("\n" + "="*60)
print("🔬 TESTING DIFFERENT ENCODERS")
print("="*60)

HDC_DIM = 4096
results = {}

for encoder_name, model_name in ENCODERS.items():
    print(f"\n{'='*60}")
    print(f"📊 Testing: {encoder_name}")
    print(f"   Model: {model_name}")
    print(f"{'='*60}")
    
    # Load encoder
    print("   Loading encoder...")
    encoder = SentenceTransformer(model_name)
    emb_dim = encoder.get_sentence_embedding_dimension()
    print(f"   Embedding dim: {emb_dim}")
    
    # Encode
    print("   Encoding sentences...")
    train_p = encoder.encode(train_premises, show_progress_bar=True)
    train_h = encoder.encode(train_hypotheses, show_progress_bar=True)
    test_p = encoder.encode(test_premises, show_progress_bar=True)
    test_h = encoder.encode(test_hypotheses, show_progress_bar=True)
    
    # Create pair features
    train_features = make_pair_features(train_p, train_h)
    test_features = make_pair_features(test_p, test_h)
    feature_dim = train_features.shape[1]
    print(f"   Feature dim: {feature_dim}")
    
    # Test all 4 configurations
    encoder_results = {}
    
    # A: Float baseline
    print("\n   Testing A: Float (no HDC)...")
    acc_A = train_and_evaluate(train_features, train_labels, 
                                test_features, test_labels, feature_dim)
    encoder_results['A_float'] = acc_A
    print(f"   ✅ A: {acc_A:.1%}")
    
    # B: Float + Projection
    print("   Testing B: Float + Projection...")
    train_proj = random_projection(train_features, HDC_DIM)
    test_proj = random_projection(test_features, HDC_DIM)
    acc_B = train_and_evaluate(train_proj, train_labels,
                                test_proj, test_labels, HDC_DIM)
    encoder_results['B_projected'] = acc_B
    print(f"   ✅ B: {acc_B:.1%}")
    
    # C: Ternary only
    print("   Testing C: Ternary (no projection)...")
    train_tern = ternary_quantize(train_features)
    test_tern = ternary_quantize(test_features)
    acc_C = train_and_evaluate(train_tern, train_labels,
                                test_tern, test_labels, feature_dim)
    encoder_results['C_ternary'] = acc_C
    print(f"   ✅ C: {acc_C:.1%}")
    
    # D: Full HDC
    print("   Testing D: Full HDC (projection + ternary)...")
    train_hdc = ternary_quantize(train_proj)
    test_hdc = ternary_quantize(test_proj)
    acc_D = train_and_evaluate(train_hdc, train_labels,
                                test_hdc, test_labels, HDC_DIM)
    encoder_results['D_full_hdc'] = acc_D
    print(f"   ✅ D: {acc_D:.1%}")
    
    encoder_results['embedding_dim'] = emb_dim
    encoder_results['feature_dim'] = feature_dim
    results[encoder_name] = encoder_results
    
    # Clean up
    del encoder
    torch.cuda.empty_cache()

In [ ]:
print("\n" + "="*60)
print("📊 RESULTS COMPARISON")
print("="*60)

# Create comparison table
print(f"\n{'Encoder':<25} {'Emb':<6} {'A:Float':<10} {'B:Proj':<10} {'C:Tern':<10} {'D:HDC':<10}")
print("-" * 75)

for encoder_name, data in results.items():
    print(f"{encoder_name:<25} {data['embedding_dim']:<6} "
          f"{data['A_float']:.1%}      {data['B_projected']:.1%}      "
          f"{data['C_ternary']:.1%}      {data['D_full_hdc']:.1%}")

In [ ]:
# Calculate improvements
baseline_A = results['MiniLM (baseline)']['A_float']
baseline_D = results['MiniLM (baseline)']['D_full_hdc']

print("\n" + "="*60)
print("📈 IMPROVEMENT OVER BASELINE")
print("="*60)

print(f"\n{'Encoder':<25} {'A vs baseline':<15} {'D vs baseline'}")
print("-" * 55)

for encoder_name, data in results.items():
    imp_A = (data['A_float'] - baseline_A) * 100
    imp_D = (data['D_full_hdc'] - baseline_D) * 100
    print(f"{encoder_name:<25} {imp_A:+.1f}%          {imp_D:+.1f}%")

In [ ]:
# Find best encoder
best_encoder = max(results.keys(), key=lambda k: results[k]['A_float'])
best_A = results[best_encoder]['A_float']
best_D = results[best_encoder]['D_full_hdc']

print("\n" + "="*60)
print("🎯 CONCLUSIONS")
print("="*60)

print(f"\n🏆 Best encoder: {best_encoder}")
print(f"   Float baseline (A): {best_A:.1%}")
print(f"   Full HDC (D): {best_D:.1%}")

if best_A > 0.65:
    print(f"\n✅ NLI encoder significantly improved baseline!")
    print(f"   Improvement: +{(best_A - baseline_A)*100:.1f}%")
    print(f"   → Encoder was the main bottleneck")
    verdict = "NLI_ENCODER_HELPS"
elif best_A > 0.58:
    print(f"\n📈 Moderate improvement with NLI encoder")
    print(f"   Improvement: +{(best_A - baseline_A)*100:.1f}%")
    print(f"   → Encoder helps, but other factors also matter")
    verdict = "PARTIAL_IMPROVEMENT"
else:
    print(f"\n⚠️ NLI encoder did not help much")
    print(f"   Improvement: +{(best_A - baseline_A)*100:.1f}%")
    print(f"   → Need different approach (two-vector, architecture change)")
    verdict = "NEED_DIFFERENT_APPROACH"

# HDC overhead
hdc_overhead = (best_A - best_D) * 100
print(f"\n📉 HDC overhead (A→D): {hdc_overhead:.1f}%")

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Comparison bars
ax = axes[0]
encoder_names = list(results.keys())
x = np.arange(len(encoder_names))
width = 0.2

configs = ['A_float', 'B_projected', 'C_ternary', 'D_full_hdc']
config_labels = ['A: Float', 'B: +Proj', 'C: Ternary', 'D: HDC']
colors = ['green', 'lightblue', 'orange', 'lightcoral']

for i, (cfg, label, color) in enumerate(zip(configs, config_labels, colors)):
    values = [results[enc][cfg] for enc in encoder_names]
    bars = ax.bar(x + i*width, values, width, label=label, color=color, edgecolor='black')

ax.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5, label='Random')
ax.set_ylabel('Accuracy')
ax.set_title('Encoder Comparison: All Configurations')
ax.set_xticks(x + 1.5*width)
ax.set_xticklabels([n.replace(' ', '\n') for n in encoder_names])
ax.legend(loc='upper left')
ax.set_ylim(0.3, max([results[e]['A_float'] for e in encoder_names]) + 0.1)

# Float baseline comparison
ax = axes[1]
float_accs = [results[enc]['A_float'] for enc in encoder_names]
hdc_accs = [results[enc]['D_full_hdc'] for enc in encoder_names]

x = np.arange(len(encoder_names))
width = 0.35

bars1 = ax.bar(x - width/2, float_accs, width, label='A: Float (ceiling)', color='green', edgecolor='black')
bars2 = ax.bar(x + width/2, hdc_accs, width, label='D: Full HDC', color='lightcoral', edgecolor='black')

ax.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Accuracy')
ax.set_title('Float Ceiling vs Full HDC')
ax.set_xticks(x)
ax.set_xticklabels([n.replace(' ', '\n') for n in encoder_names])
ax.legend()

for bar, acc in zip(bars1, float_accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{acc:.1%}', ha='center', fontweight='bold', fontsize=10)
for bar, acc in zip(bars2, hdc_accs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{acc:.1%}', ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('phase2_encoder_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save results
output = {
    'experiment': 'Phase 2: NLI-Tuned Encoder',
    'dataset': 'MNLI',
    'train_size': TRAIN_SIZE,
    'test_size': TEST_SIZE,
    'hdc_dim': HDC_DIM,
    'results': results,
    'best_encoder': best_encoder,
    'best_float_accuracy': best_A,
    'best_hdc_accuracy': best_D,
    'baseline_improvement': float(best_A - baseline_A),
    'hdc_overhead': float(best_A - best_D),
    'verdict': verdict,
    'timestamp': datetime.now().isoformat()
}

with open('phase2_encoder_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print("\n✅ Results saved!")
print(json.dumps(output, indent=2))

In [ ]:
print("\n" + "="*60)
print("📋 NEXT STEPS")
print("="*60)

if verdict == "NLI_ENCODER_HELPS":
    print("""
✅ NLI encoder is the solution!

Next steps:
1. Use best NLI encoder as default for pair tasks
2. Test on full MNLI dataset
3. Try Project-First-Bind-Later with NLI encoder
    """)
elif verdict == "PARTIAL_IMPROVEMENT":
    print("""
📈 NLI encoder helps but not enough.

Next steps:
1. Try two-vector approach (separate HDC for P and H)
2. Try Project-First-Bind-Later architecture
3. Combine NLI encoder with better architecture
    """)
else:
    print("""
⚠️ Need fundamentally different approach.

Next steps:
1. Try two-vector approach (P_hdc + H_hdc → MLP)
2. Try role-filler binding (VSA-style)
3. Consider learned projections
    """)

In [ ]:
from google.colab import files
files.download('phase2_encoder_results.json')
files.download('phase2_encoder_comparison.png')